<a href="https://colab.research.google.com/github/imbalzy/RecipeQA-FInal-Project-2470/blob/main/visual_coherent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf

class Model(tf.keras.Model):
  def __init__(self, window_sz, vocab_sz, num_images, image_shape):
    self.window_size = window_sz
    self.vocab_size = vocab_sz
    self.num_images = num_images
    self.image_shape = image_shape
    

    self.image_conv1 = tf.keras.layers.Conv2D(filters = 16, kernel_size = 3, strides=(1, 1), padding='valid')
    self.image_conv2 = tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, strides=(1, 1), padding='valid')
    self.image_conv3 = tf.keras.layers.Conv2D(filters = 64, kernel_size = 3, strides=(1, 1), padding='valid')
    self.image_dense = tf.keras.layers.dense(units = 100)

    self.word_embedding = tf.Variable(tf.random.truncated_normal([self.english_vocab_size, self.embedding_size], stddev = 0.1))
    self.encoder = tf.keras.layers.GRU(units = 100, return_sequences=True, return_state=True)
    self.text_dense = tf.keras.layers.dense(units = 100)

    self.text_image_embedding1 = tf.keras.layers.dense(units = 200, activation = "relu")
    self.text_image_embedding2 = tf.keras.layers.dense(units = 100)

    self.class_dense1 = tf.keras.layers.dense(units = 50, activation = "relu")
    self.class_dense2 = tf.keras.layers.dense(units = 20, activation = "relu")
    self.class_dense3 = tf.keras.layers.dense(units = 4)

  def call(self, query_text, choice_image):

    choice_token = self.image_conv1(choice_image[0])
    choice_token = self.image_conv2(choice_token)
    choice_token = self.image_conv3(choice_token)
    choice0_embedding = self.image_dense(choice_token)

    choice_token = self.image_conv1(choice_image[1])
    choice_token = self.image_conv2(choice_token)
    choice_token = self.image_conv3(choice_token)
    choice1_embedding = self.image_dense(choice_token)

    choice_token = self.image_conv1(choice_image[2])
    choice_token = self.image_conv2(choice_token)
    choice_token = self.image_conv3(choice_token)
    choice2_embedding = self.image_dense(choice_token)

    choice_token = self.image_conv1(choice_image[3])
    choice_token = self.image_conv2(choice_token)
    choice_token = self.image_conv3(choice_token)
    choice3_embedding = self.image_dense(choice_token)

    #query word embedding
    embedding = tf.nn.embedding_lookup(self.word_embedding, query_text)
    final_output, final_state = self.encoder(embedding, initial_state = None)
    text_embedding = self.text_dense(final_output)

    #create image_word embedding for choice 0
    token = tf.stack(choice0_embedding, text_embedding)
    token = self.text_image_embedding1(token)
    choice0_embedding = self.text_image_embedding2(token)

     #create image_word embedding for choice 1
    token = tf.stack(choice1_embedding, text_embedding)
    token = self.text_image_embedding1(token)
    choice1_embedding = self.text_image_embedding2(token)

     #create image_word embedding for choice 2
    token = tf.stack(choice2_embedding, text_embedding)
    token = self.text_image_embedding1(token)
    choice2_embedding = self.text_image_embedding2(token)

     #create image_word embedding for choice 3
    token = tf.stack(choice3_embedding, text_embedding)
    token = self.text_image_embedding1(token)
    choice3_embedding = self.text_image_embedding2(token)

    token = tf.concat(choice0_embedding, choice1_embedding, choice2_embedding, choice3_embedding)
    token = self.class_dense1(token)
    token = self.class_dense2(token)
    logit = self.class_dense3(token)
    
  def loss(self, probs, labels):
    return  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels, logits))












     



    
    
   
  